In [1]:
import numpy as np
import pandas as pd
import re
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider, Select, HoverTool, MultiChoice, CheckboxGroup, Div, Toggle, TableColumn, DataTable, RangeSlider, TextInput, Spinner
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.embed import components
from bokeh.io import output_notebook, reset_output

reset_output()
output_notebook()

Loading BokehJS ...

## Big Pile of code
Creating the Dashboard to show on the index html

In [2]:
hover = HoverTool(tooltips='''
<font color="black">
<b>Regra: <font color="#f5a91b" style="text-transform: uppercase">@r</font></b>
<br>
<b>Comunicacões</b>: @y{int}
<br>
<b>Revisões</b>: @x{int}
<br>
<b>Eficiência</b>: @e
</font>
''')

p = figure(y_range=(0, 1500), x_range=(0, 3000), plot_width=500, plot_height=300, toolbar_location=None,
           tools=[hover], title='Comunicações x Revisões - Visão Mensal')

x = np.linspace(0, 3000, 100)
y = x/2
e = list(map(lambda _: f'{_:.1f}%', 100*y/x))
r = ['linha 50%'] * 100
linha_50 = ColumnDataSource(dict(x=x, y=y, e=e, r=r))

regras = ColumnDataSource(dict(
    x=[300, 400, 500, 1000],
    y=[50, 30, 300, 1000],
    e=list(map(lambda _: f'{100*_:.1f}%', [50./300, 30./400, 300./500, 1.])),
    r=['Regra 1', 'Regra 2', 'Regra 3', 'Regra 4'],
))
regras_cpy = ColumnDataSource(dict(
    x=[300, 400, 500, 1000],
    y=[50, 30, 300, 1000],
    e=list(map(lambda _: f'{100*_:.1f}%', [50./300, 30./400, 300./500, 1.])),
    r=['Regra 1', 'Regra 2', 'Regra 3', 'Regra 4'],
))

p.line(x='x',y='y', source=linha_50, color='grey', line_dash='dashed', legend_label='50% eficiência', line_width=3, line_alpha=0.8)

p.x(x='x', y='y', source=regras_cpy, color='black', size=10)

p.legend.location = "top_left"
p.yaxis.axis_label = "# Comunicações"
p.xaxis.axis_label = "# Revisões"

title = Div(text="""<br><br><span style="font-size:15px;width:100%"><b>Análise de Eficiência dos alertas</b></span><br><br><br>""")

columns = [
    TableColumn(field="r", title="Regra"),
    TableColumn(field="x", title="Revisões"),
    TableColumn(field="y", title="Comunicações"),
    TableColumn(field="e", title="Eficiência"),
]

data_table = DataTable(source=regras_cpy, columns=columns, width=400, height=300)

ef = RangeSlider(start=0, end=100, value=(0,100), step=5, title='Eficiência (%)', width=400)
c_greater = Spinner(low=0, high=8000, value=0, step=10, title='Comunicação Maior que:', width=200)
c_less = Spinner(low=0, high=8000, value=8000, step=10, title='Comunicação Menor que:', width=200)

callback = CustomJS(args=dict(source=regras, ef=ef, c_greater=c_greater,
                                 c_less=c_less, source_cpy=regras_cpy),
                    code="""
    
        var ef_min = ef.value[0];
        var ef_max = ef.value[1];
        var c_min = c_greater.value;
        var c_max = c_less.value;
        var sdata = source.data;
        console.log(c_min, c_max);
        var data = {'r': [],
                    'x': [],
                    'y': [],
                    'e': []};

        for(var i = 0; i < sdata['x'].length; i++){
            if(((100*sdata['y'][i]/sdata['x'][i]) >= ef_min) && 
               ((100*sdata['y'][i]/sdata['x'][i]) <= ef_max) &&
               (sdata['y'][i] >= c_min) &&
               (sdata['y'][i] <= c_max)
              ){
                data['r'].push(sdata['r'][i]);
                data['x'].push(sdata['x'][i]);
                data['y'].push(sdata['y'][i]);
                data['e'].push(sdata['e'][i]);
            }
        }
    
    
        source_cpy.data = data;
""")

ef.js_on_change('value', callback)
c_greater.js_on_change('value', callback)
c_less.js_on_change('value', callback)

card_comunic = Div(text='''
<div class="card">
      <p style="color: #f5a91b;text-align: center;font-size: 30px;margin: 0;padding-top: 10px">720</p>
      <p style="text-align: center;font-size: 20px;margin: 0;padding-top: 10px;padding-bottom: 10px;">Comunicações</p>
</div>
<style>
.card {
  box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
  transition: 0.3s;
  width: 200px;
}

.card:hover {
  box-shadow: 0 8px 16px 0 rgba(0,0,0,0.2);
}
</style>
'''.strip())

card_rev = Div(text='''
<div class="card">
      <p style="color: #f5a91b;text-align: center;font-size: 30px;margin: 0;padding-top: 10px">2300</p>
      <p style="text-align: center;font-size: 20px;margin: 0;padding-top: 10px;padding-bottom: 10px;">Revisões</p>
</div>
''')

card_ef = Div(text='''
<div class="card">
      <p style="color: #f5a91b;text-align: center;font-size: 30px;margin: 0;padding-top: 10px">31,3%</p>
      <p style="text-align: center;font-size: 20px;margin: 0;padding-top: 10px;padding-bottom: 10px;">Eficiência</p>
</div>
''')

card_mes = Div(text='''
<div class="card">
      <p style="color: #f5a91b;text-align: center;font-size: 30px;margin: 0;padding-top: 10px">Março</p>
      <p style="text-align: center;font-size: 20px;margin: 0;padding-top: 10px;padding-bottom: 10px;">Mês Referência</p>
</div>
''')

layout = column(
    row(card_mes, card_comunic, card_rev, card_ef),
    title,
    row(p,
        column(ef, row(c_greater, c_less), data_table)
       )
)
show(layout)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


**Using Bokeh to create the div and javascript template, find the id and data_id given on the div**

In [3]:
script, div = components(layout)
idName = re.search(r'.* id=\"(.*?)\".*', div).group(1)
data_id = re.search(r'.*data-root-id=\"(.*?)\".*', div).group(1)
print(div, f'\n\nid: "{idName}",', f'data_id: "{data_id}"')


<div class="bk-root" id="f7c7c24f-1e90-476d-b6a4-48513d46b57b" data-root-id="1069"></div> 

id: "f7c7c24f-1e90-476d-b6a4-48513d46b57b", data_id: "1069"


**Replacing the id created for the id chosen to be on the index.html, and updating test.js**

In [4]:
with open('../scripts/test.js', 'w') as f:
    f.write(
        '\n'.join(
            script.strip()
                  .replace(idName, 'test_plot')
                  .replace(data_id, '4705925723885363430')
                  .split('\n')
                  [1:-1]
                 )
           )

**Now the index.html will update to the Dashboard created here!!**